In [ ]:
import pandas as pd
import numpy as np
import yaml
from time import sleep
from pathlib import Path

from utils import get_season, nfl_weeks_pull, game_keys_pull, reg_season, post_season
from yahoo_query import league_season_data

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]

try:
    with open(PATH) as file:
        CREDS = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

# TODAY = np.datetime64("today", "D")

# TODAY = np.datetime64("2022-01-05")
# TODAY = np.datetime64("2022-01-02")
# TODAY = np.datetime64("2021-12-28")
# TODAY = np.datetime64("2021-12-21")
# TODAY = np.datetime64("2021-12-14")
# TODAY = np.datetime64("2021-12-07")
# TODAY = np.datetime64("2021-11-30")
# TODAY = np.datetime64("2021-11-23")
# TODAY = np.datetime64("2021-11-16")
# TODAY = np.datetime64("2021-11-09")
# TODAY = np.datetime64("2021-11-02")
# TODAY = np.datetime64("2021-10-26")
# TODAY = np.datetime64("2021-10-19")
# TODAY = np.datetime64("2021-10-12")
# TODAY = np.datetime64("2021-10-05")
# TODAY = np.datetime64("2021-09-28")
# TODAY = np.datetime64("2021-09-21")
# TODAY = np.datetime64("2021-09-14")
# TODAY = np.datetime64("2021-09-09")
TODAY = np.datetime64("2021-08-31")

YEAR = TODAY.astype("datetime64[Y]").astype(int) + 1970
NFL_WEEKS = nfl_weeks_pull()
MAX_WEEK = NFL_WEEKS["week"].max()
GAME_KEYS = game_keys_pull(first="no")
# SEASON = get_season()
SEASON = 2021

try:
    LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
except Exception as e:
    print(e)
GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]

try:
    NFL_WEEK = NFL_WEEKS["week"][
        (NFL_WEEKS["end"] >= TODAY) & (NFL_WEEKS["start"] <= TODAY)
    ].values[0]

except Exception as e:
    print(e)
    NFL_WEEK = np.nan


CONSUMER_KEY = CREDS["YFPY_CONSUMER_KEY"]
CONSUMER_SECRET = CREDS["YFPY_CONSUMER_SECRET"]

league = league_season_data(
    auth_dir=PATH.parent,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)
print(TODAY)
print(NFL_WEEK)
if TODAY == np.datetime64(f"{YEAR}-08-31"):
    game_keys = league.all_game_keys()
    nfl_weeks = league.all_nfl_weeks()
    meta = league.metadata(first_time="no")
    settings, roster, stat_cat = league.set_roster_pos_stat_cat(first_time="no")
    # players = league.players_list(first_time="no")
    teams = league.teams_and_standings(first_time="no")

if (
    TODAY
    == NFL_WEEKS["end"][
        (NFL_WEEKS["week"] == MAX_WEEK) & (NFL_WEEKS["game_id"] == GAME_ID)
    ].values[0]
):
    teams = league.teams_and_standings(first_time="no")

if (
    TODAY
    == NFL_WEEKS["start"][
        (NFL_WEEKS["game_id"] == GAME_ID) & (NFL_WEEKS["week"] == 1)
    ].values[0]
):
    draft = league.draft_results(first_time="no")

if 1 <= NFL_WEEK <= 17:
    week_roster = league.team_roster_by_week(first_time="no", nfl_week=NFL_WEEK)
    matchups = league.matchups_by_week(first_time="no", nfl_week=NFL_WEEK)
    team_points = league.team_points_by_week(first_time="no", nfl_week=NFL_WEEK)
    reg_season_board = reg_season(GAME_ID, NFL_WEEK)
    post_season_board = post_season(reg_season_board, GAME_ID, NFL_WEEK)

In [3]:
pd.set_option("display.max_columns", None)